In [29]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
from sklearn.preprocessing import  MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)
from sklearn.model_selection import GridSearchCV
from collections import Counter
import json

# Model KNN (K-Nearest Neighbors)

## Zestaw danych

In [30]:
try:
    dataset = fetch_ucirepo(id=519)
    features = dataset.data.features[["age", "serum_creatinine", "ejection_fraction"]]
    target = dataset.data.targets
except Exception as e:
    df = pd.read_csv("heart_failure_clinical_records_dataset.csv")
    df.columns = [col.lower() if col == "DEATH_EVENT" else col for col in df.columns]
    features = df[["age", "serum_creatinine", "ejection_fraction"]]
    target = df["death_event"]


df = pd.concat([features, target], axis=1)
df

,age,serum_creatinine,ejection_fraction,death_event
0,75.0,1.9,20,1
1,55.0,1.1,38,1
2,65.0,1.3,20,1
3,50.0,1.9,20,1
4,65.0,2.7,20,1
...,...,...,...,...
294,62.0,1.1,38,0
295,55.0,1.2,38,0
296,45.0,0.8,60,0
297,45.0,1.4,38,0


In [31]:
counts = target.value_counts()
total = len(target)

for label, count in counts.items():
    percent = (count / total) * 100
    print(f"Liczba {'zmarłych' if label == (1,) else 'żywych'}: {count} ({percent:.2f}%)")

Liczba żywych: 203 (67.89%)
Liczba zmarłych: 96 (32.11%)


## Zastosowanie modelu

In [32]:
X = df[["age", "serum_creatinine", "ejection_fraction"]]
y = df["death_event"]

In [33]:
results = []

for i in range(100):
    # Podział danych na zbiór treningowy i testowy z random_state = iteracja, aby za każdym razem podział był inny
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=i
    )

    # Normalizacja danych (treningowych i testowych)
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Tworzenie modelu KNN
    model = KNeighborsClassifier()

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # Obliczanie metryk
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results.append(
        {
            "iteration": i,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
        }
    )

results_df = pd.DataFrame(results)

results_df

,iteration,accuracy,precision,recall,f1_score
0,0,0.744444,0.619048,0.464286,0.530612
1,1,0.777778,0.636364,0.538462,0.583333
2,2,0.755556,0.545455,0.500000,0.521739
3,3,0.766667,0.666667,0.551724,0.603774
4,4,0.677778,0.352941,0.250000,0.292683
...,...,...,...,...,...
95,95,0.733333,0.608696,0.482759,0.538462
96,96,0.777778,0.636364,0.538462,0.583333
97,97,0.700000,0.588235,0.606061,0.597015
98,98,0.766667,0.592593,0.615385,0.603774


#### Ewaluacja modelu

In [34]:
stats_results = results_df.aggregate(["min", "max", "mean"]).round(2)
stats_results = stats_results.drop(columns=["iteration"])
stats_results.transpose()

,min,max,mean
accuracy,0.61,0.81,0.73
precision,0.35,0.83,0.59
recall,0.24,0.76,0.49
f1_score,0.29,0.68,0.53


#### Optymalizacja modelu

- Wyszukiwanie najlepszych parametrów z GridSearch

Metoda GridSearchCV jest jedną z popularnych technik optymalizacji modeli regresji logistycznej, automatyzującą wyszukiwanie najlepszych hiperparametrów, takich jak siła i typ regularyzacji. Poprawia wydajność modelu poprzez użycie walidacji krzyżowej, zapewniając solidność i możliwość uogólnienia na nowe dane.

`Testowane parametry:`
- n_neighbors: określa, ile najbliższych sąsiadów uwzględnia model podczas klasyfikacji nowej próbki
- weights: określa, jak bardzo wpływają sąsiedzi na decyzję klasyfikacyjną
  - 'uniform': każdy sąsiad ma taką samą wagę
  - 'distance': bliżsi sąsiedzi mają większą wagę niż dalsi (lepiej, gdy dane mają zmienne zagęszczenie)
- metric: KNN opiera się na liczeniu odległości między próbkami, a metryka odległości określa, jak to robić
- p: stopień normy (używany przy metryce Minkowskiego)

In [35]:
results = []

for i in range(100):
    # Podział danych na zbiór treningowy i testowy z random_state = iteracja, aby za każdym razem podział był inny
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=i
    )

    # Normalizacja danych (treningowych i testowych)
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Tworzenie modelu KNN
    model = KNeighborsClassifier()

    param_grid = {
        "n_neighbors": list(range(1, 31)),
        "weights": ["uniform", "distance"],
        "metric": ["euclidean", "manhattan", "minkowski"],
        "p": [1, 2]
    }

    grid_search = GridSearchCV(
        estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, scoring="f1"
    )

    grid_search.fit(X_train, y_train)

    # Obliczanie metryk
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results.append({"best_params": grid_search.best_params_})

results_gs = pd.DataFrame(results)
results_gs

,best_params
0,"{'metric': 'euclidean', 'n_neighbors': 12, 'p'..."
1,"{'metric': 'euclidean', 'n_neighbors': 13, 'p'..."
2,"{'metric': 'manhattan', 'n_neighbors': 1, 'p':..."
3,"{'metric': 'euclidean', 'n_neighbors': 14, 'p'..."
4,"{'metric': 'euclidean', 'n_neighbors': 13, 'p'..."
...,...
95,"{'metric': 'euclidean', 'n_neighbors': 8, 'p':..."
96,"{'metric': 'manhattan', 'n_neighbors': 3, 'p':..."
97,"{'metric': 'euclidean', 'n_neighbors': 11, 'p'..."
98,"{'metric': 'euclidean', 'n_neighbors': 11, 'p'..."


In [36]:
best_params_list = [json.dumps(result["best_params"]) for result in results]

counter = Counter(best_params_list)

# Sortowanie wyników od najczęstszych do najmniej częstych
sorted_results = sorted(counter.items(), key=lambda x: x[1], reverse=True)

print("Najczęściej występujące najlepsze parametry:")
for params, count in sorted_results[:3]:
    print(f"Parametry: {params} - Wystąpienia: {count}")

Najczęściej występujące najlepsze parametry:
Parametry: {"metric": "manhattan", "n_neighbors": 3, "p": 1, "weights": "distance"} - Wystąpienia: 11
Parametry: {"metric": "euclidean", "n_neighbors": 3, "p": 1, "weights": "distance"} - Wystąpienia: 7
Parametry: {"metric": "euclidean", "n_neighbors": 11, "p": 1, "weights": "uniform"} - Wystąpienia: 7


In [37]:
results = []

for i in range(100):
    # Podział danych na zbiór treningowy i testowy z random_state = iteracja, aby za każdym razem podział był inny
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=i
    )

    # Normalizacja danych (treningowych i testowych)
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Tworzenie modelu KNN
    model = KNeighborsClassifier(metric="manhattan", p=1, n_neighbors=3, weights="distance")

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # Obliczanie metryk
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results.append(
        {
            "iteration": i,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
        }
    )

results_df = pd.DataFrame(results)

stats_results = results_df.aggregate(["min", "max", "mean"]).round(2)
stats_results = stats_results.drop(columns=["iteration"])
stats_results.transpose()

,min,max,mean
accuracy,0.64,0.87,0.73
precision,0.35,0.79,0.59
recall,0.38,0.76,0.57
f1_score,0.44,0.76,0.58
